<a href="https://colab.research.google.com/github/kimdongha723/kdh723/blob/main/BTC_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyupbit

In [19]:
import pyupbit

In [20]:
#BTC 최근 200시간의 데이터 불러옴
df = pyupbit.get_ohlcv("KRW-BTC", interval="minute60")
df

,open,high,low,close,volume,value
2023-10-25 08:00:00,45849000.0,45939000.0,45423000.0,45599000.0,251.115103,1.146500e+10
2023-10-25 09:00:00,45593000.0,45872000.0,45556000.0,45863000.0,311.503416,1.423390e+10
2023-10-25 10:00:00,45863000.0,46330000.0,45850000.0,46032000.0,516.720223,2.382697e+10
2023-10-25 11:00:00,45999000.0,46030000.0,45764000.0,45850000.0,186.543476,8.561337e+09
2023-10-25 12:00:00,45864000.0,45986000.0,45810000.0,45885000.0,157.762308,7.239769e+09
...,...,...,...,...,...,...
2023-11-02 11:00:00,48262000.0,48661000.0,48186000.0,48428000.0,451.394573,2.185375e+10
2023-11-02 12:00:00,48417000.0,48594000.0,48315000.0,48315000.0,273.577870,1.324950e+10
2023-11-02 13:00:00,48315000.0,48350000.0,48152000.0,48177000.0,179.689088,8.673380e+09
2023-11-02 14:00:00,48178000.0,48178000.0,47829000.0,47928000.0,408.138538,1.957893e+10


In [21]:
#시간(ds)와 종가(y)값만 남김
df = df.reset_index()
df['ds'] = df['index']
df['y'] = df['close']
data = df[['ds','y']]
data

,ds,y
0,2023-10-25 08:00:00,45599000.0
1,2023-10-25 09:00:00,45863000.0
2,2023-10-25 10:00:00,46032000.0
3,2023-10-25 11:00:00,45850000.0
4,2023-10-25 12:00:00,45885000.0
...,...,...
195,2023-11-02 11:00:00,48428000.0
196,2023-11-02 12:00:00,48315000.0
197,2023-11-02 13:00:00,48177000.0
198,2023-11-02 14:00:00,47928000.0


In [22]:
#prophet 불러옴
from fbprophet import Prophet

ModuleNotFoundError: ignored

In [ ]:
#학습
model = Prophet()
model.fit(data)

In [ ]:
#24시간 미래 예측
future = model.make_future_dataframe(periods=24, freq='H')
forecast = model.predict(future)

In [ ]:
#그래프1
fig1 = model.plot(forecast)

In [ ]:
#그래프2
fig2 = model.plot_components(forecast)

In [ ]:
#매수 시점의 가격
nowValue = pyupbit.get_current_price("KRW-BTC")
nowValue

57098000.0

In [ ]:
#종가의 가격을 구함

#현재 시간이 자정 이전
closeDf = forecast[forecast['ds'] == forecast.iloc[-1]['ds'].replace(hour=9)]

#현재 시간이 자정 이후
if len(closeDf) == 0:
  closeDf = forecast[forecast['ds'] == data.iloc[-1]['ds'].replace(hour=9)]

#어쨋든 당일 종가
closeValue = closeDf['yhat'].values[0]
closeValue

58097882.673339084

In [ ]:
#구체적인 가격
print("현재 시점 가격: ", nowValue)
print("종가의 가격: ", closeValue)

현재 시점 가격:  57098000.0
종가의 가격:  58097882.673339084


In [ ]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2021-05-08 20:00:00,7.049319e+07,6.978264e+07,7.145666e+07,7.049319e+07,7.049319e+07,73763.938017,73763.938017,73763.938017,73763.938017,73763.938017,73763.938017,0.0,0.0,0.0,7.056695e+07
1,2021-05-08 21:00:00,7.054058e+07,6.981790e+07,7.151720e+07,7.054058e+07,7.054058e+07,155154.366642,155154.366642,155154.366642,155154.366642,155154.366642,155154.366642,0.0,0.0,0.0,7.069573e+07
2,2021-05-08 22:00:00,7.058797e+07,6.993552e+07,7.159752e+07,7.058797e+07,7.058797e+07,200297.460741,200297.460741,200297.460741,200297.460741,200297.460741,200297.460741,0.0,0.0,0.0,7.078827e+07
3,2021-05-08 23:00:00,7.063536e+07,6.995876e+07,7.160555e+07,7.063536e+07,7.063536e+07,127033.847973,127033.847973,127033.847973,127033.847973,127033.847973,127033.847973,0.0,0.0,0.0,7.076239e+07
4,2021-05-09 00:00:00,7.068275e+07,6.977696e+07,7.145736e+07,7.068275e+07,7.068275e+07,-64004.026527,-64004.026527,-64004.026527,-64004.026527,-64004.026527,-64004.026527,0.0,0.0,0.0,7.061875e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2021-05-17 23:00:00,5.741650e+07,5.620334e+07,5.893053e+07,5.640791e+07,5.859450e+07,127033.847977,127033.847977,127033.847977,127033.847977,127033.847977,127033.847977,0.0,0.0,0.0,5.754353e+07
220,2021-05-18 00:00:00,5.735429e+07,5.587900e+07,5.883842e+07,5.624516e+07,5.862328e+07,-64004.026533,-64004.026533,-64004.026533,-64004.026533,-64004.026533,-64004.026533,0.0,0.0,0.0,5.729029e+07
221,2021-05-18 01:00:00,5.729208e+07,5.553360e+07,5.871707e+07,5.610166e+07,5.864511e+07,-264151.046571,-264151.046571,-264151.046571,-264151.046571,-264151.046571,-264151.046571,0.0,0.0,0.0,5.702793e+07
222,2021-05-18 02:00:00,5.722988e+07,5.527597e+07,5.845177e+07,5.594232e+07,5.868056e+07,-331244.053737,-331244.053737,-331244.053737,-331244.053737,-331244.053737,-331244.053737,0.0,0.0,0.0,5.689863e+07
